# Code Generation as a Dual Task of Code Summarization

<img src='https://i.imgur.com/RqN1agC.png' width='600' align='left'>

## References
- https://www.tensorflow.org/tutorials/text/nmt_with_attention
- https://blog.floydhub.com/attention-mechanism/
- https://github.com/Bolin0215/CSCGDual

**Dataset:** https://github.com/wanyao1992/code_summarization_public

## Definitions

$x \; \text{: code snippets}, \; y \; \text{: comments}$

$P(x,y) = \color{#00a010}{P(x) \cdot P(y|x)} = \color{#a010a0}{P(y) \cdot P(x|y)}$

### Loss terms

$l_{xy} = -\frac{1}{m} \sum_{t=1}^{m} P(y_t | y_{\lt t}, x)$

$l_{yx} = -\frac{1}{n} \sum_{t=1}^{n} P(x_t | x_{\lt t}, y)$

$l_{dual}=\left[ \left(\color{#00a010}{\log\hat{P}(x) + \log P(y \vert x; \theta_{xy})} \right) - \left(\color{#a010a0}{\log\hat{P}(y) + \log P(x \vert y; \theta_{yx})} \right) \right]^{2} \text{ : regularization term}$

$l_{att} = l_1 + l_2, \text{ where } l_k = \mathcal{D}_{JS} \left( b_i, b_i' \right ) = \frac{1}{2n}\sum_{i=1}^{n} \mathcal{D}_{KL} \left(b_i \, || \, \frac{b_i + b_i'}{2} \right) + \mathcal{D}_{KL} \left(b_i' \, || \, \frac{b_i + b_i'}{2} \right)$

$b_i = softmax \left( A_{xy}[i, :] \right), \; b_i' = softmax \left( A_{yx}[i, :] \right)$

$A_{xy} \in \mathbb{R}^{n \times m}, \; A_{yx} \in \mathbb{R}^{m \times n} \text{ : attention weights}$

### Updates

$\text{Minibatch of } k \text{ pairs: } \langle \left(x_i, y_i\right) \rangle_{i=1}^{k}$

$
\begin{cases}
G_{xy} = \nabla_{\theta_{xy}} \frac{1}{k} \sum_{i=1}^{k} \left( l_{xy} + \lambda_{dual}^{(1)} \cdot l_{dual} + \lambda_{att}^{(1)} \cdot l_{att} \right)\\
G_{yx} = \nabla_{\theta_{yx}} \frac{1}{k} \sum_{i=1}^{k} \left( l_{yx} + \lambda_{dual}^{(2)} \cdot l_{dual} + \lambda_{att}^{(2)} \cdot l_{att} \right)
\end{cases}
$

$\text{Update } \theta_{xy} \text{ and } \theta_{yx} \text{ independently}$

### Notes
- The last encoder's hidden state is used to init the decoder's hidden state.

### Hyperparameters

In [ ]:
import os
from argparse import Namespace

import numpy as np
import pandas as pd

from timeit import default_timer as timer
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as O
from torch.utils.data import DataLoader

from datasets import Django

### Globals

In [ ]:
EMB_DIR    = '/home/alex/workspace/msc-research/embeddings'
DJANGO_DIR = '/home/alex/workspace/msc-research/raw-datasets/django/'

### Hyperparameters

In [ ]:
HP = Namespace()
HP.batch_size = 5
HP.epochs     = 1

HP.dataset_config = Namespace()
HP.dataset_config.__dict__ = {
    'root_dir': DJANGO_DIR,
    'anno_min_freq': 1,
    'code_min_freq': 1,
    'anno_seq_maxlen': 40,
    'code_seq_maxlen': 40,
    'emb_file': os.path.join(EMB_DIR, 'glove.6B.50d.txt.pickle')
}

django = Django(config=HP.dataset_config)

### Encoder

In [ ]:
class Encoder(nn.Module):
    def __init__(self, hidden_size, input_maxlen, emb_matrix, num_layers=1, bidir=True):
        super(Encoder, self).__init__()
        
        self.hidden_size  = hidden_size
        self.input_maxlen = input_maxlen
        
        self.num_layers = num_layers
        self.bidir = bidir
        
        self.vocab_size, self.emb_dim = emb_matrix.shape
        
        self.embedding = nn.Embedding(self.vocab_size, self.emb_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(emb_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        
        self.bidir_lstm = nn.LSTM(input_size=self.emb_dim,
                                  hidden_size=self.hidden_size,
                                  num_layers=self.num_layers,
                                  bidirectional=self.bidir,
                                  batch_first=True)
        
    def forward(self, x, hidden=None):
        if hidden is None:
            hidden = self.init_hidden(batch_size=x.shape[0])
            
        emb = self.embedding(x)
        out, hidden = self.bidir_lstm(emb)
            
        return out, hidden
    
    def init_hidden(self, batch_size):
        d = 2 if self.bidir else 1
        z = torch.zeros(d * self.num_layers, batch_size, self.hidden_size)
        return (z,z)

In [ ]:
HS = 512
L = 40
enc = Encoder(hidden_size=HS, 
              input_maxlen=L, 
              emb_matrix=django.anno_lang.emb_matrix, 
              bidir=True, 
              num_layers=1)

print(enc)

bs = 17
x = torch.randint(L, size=(bs, L))
out, (h, c) = enc(x)
print('out:', out.shape, 'h:', h.shape, 'c:', c.shape)

h = h.permute((1, 0, 2)).reshape(bs, -1)
# torch.isclose(out[:, -1], h)
assert torch.isclose(out[:, -1, :HS], h[:, :HS]).all()
assert torch.isclose(out[:, 0, HS:], h[:, HS:]).all()

### Luong Attention

In [ ]:
class LuongAttention(nn.Module):
    def __init__(self, enc_hidden_size, dec_hidden_size):
        super(LuongAttention, self).__init__()
        
        self.W = nn.Linear(enc_hidden_size, dec_hidden_size)
        
    def forward(self, ht, hs):
        # ht: decoder output: (batch_size, 1, hidden_size)
        # hs: encoder output: (batch_size, seq_len, hidden_size)
        # score: (batch_size, 1, seq_len)
        
        score   = torch.bmm(ht, self.W(hs).transpose(1, 2))
        align   = F.softmax(score, dim=-1)
        context = torch.bmm(align, hs)
        
        return context, align

In [ ]:
att = LuongAttention(2*HS, HS)
print(att)
d = torch.rand(bs, 1, HS)
c, a = att(d, out)
print('context', c.shape)
print('align', a.shape)

### Decoder

- maybe no code embeddings?
- Coarse-to-Fine does not use code embeddings (only input + sketch embeddings)

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, enc_hidden_size, dec_hidden_size, num_layers=1, bidir=False):
        super(Decoder, self).__init__()
        
        self.emb_dim = emb_dim
        self.hidden_size = dec_hidden_size
        
        self.embedding = nn.Embedding(vocab_size, emb_dim) # one hot
        
        self.num_layers = num_layers
        self.bidir = bidir
        
        self.lstm = nn.LSTM(input_size=self.emb_dim,
                            hidden_size=self.hidden_size,
                            num_layers=self.num_layers,
                            bidirectional=self.bidir,
                            batch_first=True)
        
#         self.linear = nn.Linear()
        
        self.attention = LuongAttention(enc_hidden_size, self.hidden_size)
        
    def forward(self, x, hidden, enc_output):
        context, align = self.attention(hidden, enc_output)
        
        emb = self.embedding(x)
        
        lstm_in = torch.cat((emb, context), dim=-1)
        
        output, hidden = self.lstm(lstm_in, hidden)
        
        return output, hidden

In [ ]:
dec

## Training

In [ ]:
x = torch.randint(1000, size=(1,40))

enc = Encoder(hidden_size=512, 
              input_maxlen=40, 
              emb_matrix=django.anno_lang.emb_matrix, 
              bidir=True, 
              num_layers=1)

dec = Decoder(1000, 50, 1024, 512, 1, False)
out, (h, c) = enc(x)

inp = torch.randint(40, size=(17, 50))
output, hidden = dec(inp, h, out)

In [ ]:
# print(django.raw_example(1337))

# s = []
# for i in django.X[1337]:
#     s += [django.anno_lang.index2word[i]]
# print(' '.join(s))

# s = []
# for i in django.Y[1337]:
#     s += [django.code_lang.index2word[i]]
# print(' '.join(s))

In [ ]:
from utils import load_pt_glove, build_embedding_matrix

emb_dict = load_pt_glove(os.path.join(EMB_DIR, 'glove.6B.50d.txt.pickle'))
e1 = django.anno_lang.emb_matrix

In [ ]:
for w in 'this is a test return if self object'.split():
    assert np.isclose(emb_dict[w], e1[django.anno_lang.word2index[w]]).all()

In [ ]:
x, y = django[12]
print(x)
print(y)
django.raw_example(12)
django.anno_lang

In [ ]:
django.code_lang.word2count.most_common()[-100:]